In [9]:
import sagemaker
import numpy as np
import pandas as pd
import os
import torch
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [10]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

## Deploy model

In [71]:
class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data='s3://sagemaker-eu-west-1-087816224558/sagemaker-pytorch-2020-05-31-18-26-11-789/output/model.tar.gz',
                     role = role,
                     framework_version='1.1.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)

In [72]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------------!

In [86]:
tag2name = {14: 'B-art',
                16: 'B-eve',
                0: 'B-geo',
                13: 'B-gpe',
                12: 'B-nat',
                10: 'B-org',
                4: 'B-per',
                2: 'B-tim',
                5: 'I-art',
                7: 'I-eve',
                15: 'I-geo',
                8: 'I-gpe',
                11: 'I-nat',
                3: 'I-org',
                6: 'I-per',
                1: 'I-tim',
                17: 'X',
                9: 'O',
                18: '[CLS]',
                19: '[SEP]'} 

In [ ]:
text = 'I live in USA this is my IBM laptop'

In [73]:
predictions = predictor.predict(text)

In [74]:
print(predictions)

b'[18, 9, 9, 9, 0, 9, 18, 9, 10, 9, 19, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]'


In [83]:
pred_array = np.fromstring(predictions[1:-1], dtype=np.int, sep=', ')

In [97]:
for i in zip(pred_array[1:len(text.split( ))].tolist(), text.split()):
    print(tag2name[i[0]],'------', i[1])

O ------ I
O ------ live
O ------ in
B-geo ------ USA
O ------ this
[CLS] ------ is
O ------ my
B-org ------ IBM


### Create predictor from endpoint

In [ ]:
# from sagemaker.pytorch import PyTorchPredictor
# predictor = PyTorchPredictor('sagemaker-pytorch-2020-06-01-18-56-03-855',ContentType = 'text/plain')